In [3]:
# Install required packages
import subprocess
import sys

packages = ['catboost']
for package in packages:
    try:
        __import__(package)
        print(f"✓ {package} is already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--user", package])
        print(f"✓ {package} installed successfully")

Installing catboost...
✓ catboost installed successfully
✓ catboost installed successfully


# NFL Big Data Bowl 2026 - Advanced Player Trajectory Prediction

## Objective
Predict future player positions (x, y coordinates) on the field using tracking data from the 2023 NFL season.

## Key Improvements Over Baseline:
1. **Advanced Feature Engineering**: 
   - Enhanced physics-based features (momentum, energy, force projections)
   - Temporal patterns and trajectory curvature
   - Player interaction features
   - Ball-relative coordinate system
   
2. **Multi-Model Ensemble**:
   - LightGBM + XGBoost + CatBoost
   - Role-specific models (Targeted Receiver, Defensive Coverage)
   - Residual learning approach
   
3. **Better Validation**:
   - Time-aware cross-validation
   - Horizon-weighted loss
   - Group-based splitting to prevent leakage

4. **Optimization**:
   - GPU acceleration
   - Parallel data loading
   - Memory-efficient processing

In [7]:
# ============================================================
# NFL Big Data Bowl 2026 — Enhanced Multi-Model Approach
# ============================================================

import os, warnings, math, pickle
import numpy as np
import pandas as pd
from pathlib import Path
from multiprocessing import Pool as MP, cpu_count
from tqdm.auto import tqdm
from sklearn.model_selection import GroupKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import gc

# Gradient Boosting libraries
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, Pool as CatPool
try:
    from catboost.utils import get_gpu_device_count
except:
    def get_gpu_device_count(): return 0

warnings.filterwarnings("ignore")

# Configuration
BASEDIR = Path(r"d:\ML Practice\NFL Big data\nfl-big-data-bowl-2026-prediction")
SAVE_DIR = Path(r"d:\ML Practice\NFL Big data")

N_WEEKS = 18
SEED = 42
np.random.seed(SEED)

# Cross-validation settings
N_FOLDS = 5
USE_GROUP_KFOLD = True

# Model hyperparameters
LGBM_PARAMS = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 127,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'max_depth': 10,
    'min_child_samples': 20,
    'reg_alpha': 0.1,
    'reg_lambda': 10.0,
    'verbose': -1,
    'device': 'gpu' if get_gpu_device_count() > 0 else 'cpu',
    'gpu_platform_id': 0,
    'gpu_device_id': 0,
}

XGB_PARAMS = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 9,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 3,
    'reg_alpha': 0.1,
    'reg_lambda': 10.0,
    'tree_method': 'gpu_hist' if get_gpu_device_count() > 0 else 'hist',
    'predictor': 'gpu_predictor' if get_gpu_device_count() > 0 else 'cpu_predictor',
    'seed': SEED,
}

CAT_PARAMS = {
    'iterations': 15000,
    'learning_rate': 0.08,
    'depth': 8,
    'l2_leaf_reg': 8.0,
    'task_type': 'GPU' if get_gpu_device_count() > 0 else 'CPU',
    'loss_function': 'RMSE',
    'early_stopping_rounds': 500,
    'verbose': 200,
    'random_seed': SEED,
    'border_count': 254,
}

# Feature engineering parameters
K_NEIGHBORS = 8  # Number of neighbors for interaction features
RADIUS = 35.0    # Interaction radius in yards
HORIZON_WEIGHT = 0.3  # Weight factor for longer horizons

print(f"GPU Available: {get_gpu_device_count() > 0}")
print(f"Working Directory: {SAVE_DIR}")
print(f"Data Directory: {BASEDIR}")

GPU Available: True
Working Directory: d:\ML Practice\NFL Big data
Data Directory: d:\ML Practice\NFL Big data\nfl-big-data-bowl-2026-prediction


In [8]:
def load_week(week_num: int):
    """Load input and output data for a specific week"""
    fin = BASEDIR / f"train/input_2023_w{week_num:02d}.csv"
    fout = BASEDIR / f"train/output_2023_w{week_num:02d}.csv"
    return pd.read_csv(fin), pd.read_csv(fout)

def load_all_train():
    """Load all training data in parallel"""
    print("Loading training data...")
    with MP(min(cpu_count(), N_WEEKS)) as pool:
        res = list(tqdm(pool.imap(load_week, range(1, N_WEEKS+1)), total=N_WEEKS, desc="Loading weeks"))
    
    train_input = pd.concat([r[0] for r in res], ignore_index=True)
    train_output = pd.concat([r[1] for r in res], ignore_index=True)
    
    print(f"Train input shape:  {train_input.shape}")
    print(f"Train output shape: {train_output.shape}")
    print(f"Memory usage: {train_input.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    return train_input, train_output

# Utility functions
def to_inches(height_str):
    """Convert height string (e.g., '6-1') to inches"""
    try:
        feet, inches = str(height_str).split("-")
        return float(feet) * 12.0 + float(inches)
    except:
        return np.nan

In [9]:
def engineer_physics_features(df: pd.DataFrame) -> pd.DataFrame:
    """Create physics-based features"""
    df = df.copy()
    
    # Ensure required columns exist with defaults
    for col in ['x', 'y', 's', 'a', 'o', 'dir', 'ball_land_x', 'ball_land_y']:
        if col not in df.columns:
            df[col] = 0.0
    
    # Player physical attributes
    df['height_inches'] = df.get('player_height', '6-0').apply(to_inches).fillna(72.0)
    df['weight_lbs'] = pd.to_numeric(df.get('player_weight', 200), errors='coerce').fillna(200.0)
    df['bmi'] = (df['weight_lbs'] / (df['height_inches']**2 + 1e-6)) * 703.0
    
    # Direction and heading (dir=0° points to +y axis)
    dir_rad = np.radians(pd.to_numeric(df['dir'], errors='coerce').fillna(0.0))
    df['heading_x'] = np.sin(dir_rad)
    df['heading_y'] = np.cos(dir_rad)
    
    # Velocity components
    speed = pd.to_numeric(df['s'], errors='coerce').fillna(0.0)
    accel = pd.to_numeric(df['a'], errors='coerce').fillna(0.0)
    df['velocity_x'] = speed * df['heading_x']
    df['velocity_y'] = speed * df['heading_y']
    df['acceleration_x'] = accel * df['heading_x']
    df['acceleration_y'] = accel * df['heading_y']
    
    # Ball-relative geometry
    pos_x = pd.to_numeric(df['x'], errors='coerce').fillna(0.0)
    pos_y = pd.to_numeric(df['y'], errors='coerce').fillna(0.0)
    ball_x = pd.to_numeric(df['ball_land_x'], errors='coerce').fillna(0.0)
    ball_y = pd.to_numeric(df['ball_land_y'], errors='coerce').fillna(0.0)
    
    dx = ball_x - pos_x
    dy = ball_y - pos_y
    dist = np.sqrt(dx**2 + dy**2)
    
    df['dist_to_ball'] = dist
    df['angle_to_ball'] = np.arctan2(dy, dx)
    
    # Ball-frame coordinate system (parallel and perpendicular to ball direction)
    unit_x = dx / (dist + 1e-6)
    unit_y = dy / (dist + 1e-6)
    perp_x = -unit_y
    perp_y = unit_x
    
    # Velocity projections in ball frame
    df['v_parallel'] = df['velocity_x'] * unit_x + df['velocity_y'] * unit_y
    df['v_perpendicular'] = df['velocity_x'] * perp_x + df['velocity_y'] * perp_y
    
    # Acceleration projections in ball frame
    df['a_parallel'] = df['acceleration_x'] * unit_x + df['acceleration_y'] * unit_y
    df['a_perpendicular'] = df['acceleration_x'] * perp_x + df['acceleration_y'] * perp_y
    
    # Heading alignment with ball direction
    df['heading_alignment'] = df['heading_x'] * unit_x + df['heading_y'] * unit_y
    
    # Advanced physics
    df['speed_squared'] = speed**2
    df['accel_magnitude'] = np.sqrt(df['acceleration_x']**2 + df['acceleration_y']**2)
    df['momentum_x'] = df['weight_lbs'] * df['velocity_x']
    df['momentum_y'] = df['weight_lbs'] * df['velocity_y']
    df['momentum_magnitude'] = np.sqrt(df['momentum_x']**2 + df['momentum_y']**2)
    df['kinetic_energy'] = 0.5 * df['weight_lbs'] * df['speed_squared']
    
    # Orientation features
    orient_rad = np.radians(pd.to_numeric(df['o'], errors='coerce').fillna(0.0))
    df['orient_x'] = np.sin(orient_rad)
    df['orient_y'] = np.cos(orient_rad)
    df['orient_ball_align'] = df['orient_x'] * unit_x + df['orient_y'] * unit_y
    
    # Angle differences
    df['dir_orient_diff'] = np.abs(df['dir'] - df['o'])
    df['dir_orient_diff'] = np.minimum(df['dir_orient_diff'], 360 - df['dir_orient_diff'])
    
    return df

In [10]:
def add_temporal_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add lag features, rolling statistics, and trajectory patterns"""
    df = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id']).copy()
    group_cols = ['game_id', 'play_id', 'nfl_id']
    
    # Lag features (previous timesteps)
    lag_features = ['x', 'y', 'velocity_x', 'velocity_y', 's', 'a', 
                    'v_parallel', 'v_perpendicular', 'a_parallel', 'a_perpendicular']
    
    for lag in [1, 2, 3, 5]:
        for col in lag_features:
            if col in df.columns:
                df[f'{col}_lag{lag}'] = df.groupby(group_cols)[col].shift(lag)
    
    # Rolling statistics (smoothed trends)
    rolling_features = ['x', 'y', 'velocity_x', 'velocity_y', 's', 'v_parallel', 'v_perpendicular']
    
    for window in [3, 5, 7]:
        for col in rolling_features:
            if col in df.columns:
                rolling = df.groupby(group_cols)[col].rolling(window, min_periods=1)
                df[f'{col}_rolling_mean_{window}'] = rolling.mean().reset_index(level=[0,1,2], drop=True)
                df[f'{col}_rolling_std_{window}'] = rolling.std().reset_index(level=[0,1,2], drop=True)
    
    # Delta features (changes between frames)
    delta_features = ['velocity_x', 'velocity_y', 'v_parallel', 'v_perpendicular', 
                      's', 'a', 'dist_to_ball']
    
    for col in delta_features:
        if col in df.columns:
            df[f'{col}_delta'] = df.groupby(group_cols)[col].diff()
            df[f'{col}_delta2'] = df.groupby(group_cols)[f'{col}_delta'].diff()  # second derivative
    
    # Trajectory curvature (change in direction)
    if 'angle_to_ball' in df.columns:
        df['angle_to_ball_delta'] = df.groupby(group_cols)['angle_to_ball'].diff()
    
    # Acceleration patterns
    if 'accel_magnitude' in df.columns:
        df['accel_jerk'] = df.groupby(group_cols)['accel_magnitude'].diff()  # rate of change of acceleration
    
    return df

In [11]:
def add_formation_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add team formation and spatial relationship features"""
    df = df.copy()
    
    # Team-level statistics
    grp = df.groupby(['game_id', 'play_id', 'frame_id', 'player_side'], sort=False)
    
    df['team_centroid_x'] = grp['x'].transform('mean')
    df['team_centroid_y'] = grp['y'].transform('mean')
    df['team_spread_x'] = grp['x'].transform('std').fillna(0.0)
    df['team_spread_y'] = grp['y'].transform('std').fillna(0.0)
    df['team_width'] = grp['y'].transform(lambda x: x.max() - x.min()).fillna(0.0)
    df['team_length'] = grp['x'].transform(lambda x: x.max() - x.min()).fillna(0.0)
    
    # Player position relative to team centroid
    df['rel_centroid_x'] = df['x'] - df['team_centroid_x']
    df['rel_centroid_y'] = df['y'] - df['team_centroid_y']
    df['dist_from_centroid'] = np.sqrt(df['rel_centroid_x']**2 + df['rel_centroid_y']**2)
    
    # Formation orientation (team bearing toward ball)
    ball_x = pd.to_numeric(df['ball_land_x'], errors='coerce').fillna(0.0)
    ball_y = pd.to_numeric(df['ball_land_y'], errors='coerce').fillna(0.0)
    
    bearing = np.arctan2(ball_y - df['team_centroid_y'], ball_x - df['team_centroid_x'])
    df['formation_bearing_sin'] = np.sin(bearing)
    df['formation_bearing_cos'] = np.cos(bearing)
    
    # Density features (how clustered is the team)
    df['team_compactness'] = df['team_spread_x'] * df['team_spread_y']
    
    return df

In [12]:
def compute_neighbor_features(df: pd.DataFrame, k_neighbors: int = K_NEIGHBORS, 
                             radius: float = RADIUS) -> pd.DataFrame:
    """Compute features based on nearby players (allies and opponents)"""
    
    cols_needed = ['game_id', 'play_id', 'nfl_id', 'frame_id', 'x', 'y',
                   'velocity_x', 'velocity_y', 'player_side']
    
    src = df[cols_needed].copy()
    
    # Get last frame for each player
    last_frame = (src.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
                     .groupby(['game_id', 'play_id', 'nfl_id'], as_index=False)
                     .tail(1)
                     .rename(columns={'frame_id': 'last_frame_id'})
                     .reset_index(drop=True))
    
    # Create neighbor pairs
    neighbors = last_frame.merge(
        src.rename(columns={
            'frame_id': 'nb_frame_id', 'nfl_id': 'nfl_id_nb', 
            'x': 'x_nb', 'y': 'y_nb',
            'velocity_x': 'vx_nb', 'velocity_y': 'vy_nb',
            'player_side': 'player_side_nb'
        }),
        left_on=['game_id', 'play_id', 'last_frame_id'],
        right_on=['game_id', 'play_id', 'nb_frame_id'],
        how='left'
    )
    
    # Remove self-pairs
    neighbors = neighbors[neighbors['nfl_id_nb'] != neighbors['nfl_id']]
    
    # Calculate distances and relative velocities
    neighbors['dx'] = neighbors['x_nb'] - neighbors['x']
    neighbors['dy'] = neighbors['y_nb'] - neighbors['y']
    neighbors['dvx'] = neighbors['vx_nb'] - neighbors['velocity_x']
    neighbors['dvy'] = neighbors['vy_nb'] - neighbors['velocity_y']
    neighbors['dist'] = np.sqrt(neighbors['dx']**2 + neighbors['dy']**2)
    
    # Filter by distance
    neighbors = neighbors[np.isfinite(neighbors['dist']) & (neighbors['dist'] > 1e-6)]
    if radius is not None:
        neighbors = neighbors[neighbors['dist'] <= radius]
    
    # Identify allies vs opponents
    neighbors['is_ally'] = (neighbors['player_side_nb'].fillna('') == 
                            neighbors['player_side'].fillna('')).astype(np.float32)
    
    # Rank neighbors by distance
    keys = ['game_id', 'play_id', 'nfl_id']
    neighbors['rank'] = neighbors.groupby(keys)['dist'].rank(method='first')
    
    if k_neighbors is not None:
        neighbors = neighbors[neighbors['rank'] <= k_neighbors]
    
    # Weighted aggregation (closer neighbors have more weight)
    tau = 8.0  # decay parameter
    neighbors['weight'] = np.exp(-neighbors['dist'] / tau)
    sum_weight = neighbors.groupby(keys)['weight'].transform('sum')
    neighbors['weight_norm'] = np.where(sum_weight > 0, neighbors['weight'] / sum_weight, 0.0)
    
    neighbors['weight_ally'] = neighbors['weight_norm'] * neighbors['is_ally']
    neighbors['weight_opp'] = neighbors['weight_norm'] * (1.0 - neighbors['is_ally'])
    
    # Weighted features
    for col in ['dx', 'dy', 'dvx', 'dvy']:
        neighbors[f'{col}_ally_w'] = neighbors[col] * neighbors['weight_ally']
        neighbors[f'{col}_opp_w'] = neighbors[col] * neighbors['weight_opp']
    
    # Separate distances for allies and opponents
    neighbors['dist_ally'] = np.where(neighbors['is_ally'] > 0.5, neighbors['dist'], np.nan)
    neighbors['dist_opp'] = np.where(neighbors['is_ally'] < 0.5, neighbors['dist'], np.nan)
    
    # Aggregate neighbor features
    agg_features = neighbors.groupby(keys).agg(
        neighbor_ally_dx_mean=('dx_ally_w', 'sum'),
        neighbor_ally_dy_mean=('dy_ally_w', 'sum'),
        neighbor_ally_dvx_mean=('dvx_ally_w', 'sum'),
        neighbor_ally_dvy_mean=('dvy_ally_w', 'sum'),
        neighbor_opp_dx_mean=('dx_opp_w', 'sum'),
        neighbor_opp_dy_mean=('dy_opp_w', 'sum'),
        neighbor_opp_dvx_mean=('dvx_opp_w', 'sum'),
        neighbor_opp_dvy_mean=('dvy_opp_w', 'sum'),
        neighbor_ally_count=('is_ally', 'sum'),
        neighbor_opp_count=('is_ally', lambda s: float(len(s) - s.sum())),
        neighbor_ally_dist_min=('dist_ally', 'min'),
        neighbor_ally_dist_mean=('dist_ally', 'mean'),
        neighbor_opp_dist_min=('dist_opp', 'min'),
        neighbor_opp_dist_mean=('dist_opp', 'mean'),
    ).reset_index()
    
    # Get distances to closest 3 neighbors
    nearest = neighbors[neighbors['rank'] <= 3][keys + ['rank', 'dist']].copy()
    nearest['rank'] = nearest['rank'].astype(int)
    nearest_wide = nearest.pivot_table(index=keys, columns='rank', values='dist', aggfunc='first')
    nearest_wide = nearest_wide.rename(columns={1: 'neighbor_d1', 2: 'neighbor_d2', 3: 'neighbor_d3'}).reset_index()
    
    agg_features = agg_features.merge(nearest_wide, on=keys, how='left')
    
    # Fill NaN values
    for col in agg_features.columns:
        if col.startswith('neighbor_'):
            if 'count' in col:
                agg_features[col] = agg_features[col].fillna(0.0)
            elif 'dist' in col or 'd1' in col or 'd2' in col or 'd3' in col:
                agg_features[col] = agg_features[col].fillna(radius if radius else 35.0)
            else:
                agg_features[col] = agg_features[col].fillna(0.0)
    
    return agg_features

In [13]:
def add_role_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add role-specific binary indicators"""
    df = df.copy()
    
    # Player role indicators
    player_role = df.get('player_role', '').astype(str)
    player_side = df.get('player_side', '').astype(str)
    
    df['role_targeted_receiver'] = (player_role == 'Targeted Receiver').astype(np.int8)
    df['role_defensive_coverage'] = (player_role == 'Defensive Coverage').astype(np.int8)
    df['role_passer'] = (player_role == 'Passer').astype(np.int8)
    df['side_offense'] = (player_side == 'Offense').astype(np.int8)
    df['side_defense'] = (player_side == 'Defense').astype(np.int8)
    
    return df

In [14]:
def create_training_rows(input_df: pd.DataFrame, output_df: pd.DataFrame) -> pd.DataFrame:
    """
    Merge input features (last observed frame) with output targets (future frames).
    Each row represents a prediction task: given last observed state, predict future position.
    """
    # Get last frame observation for each player
    last_observations = (
        input_df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
                .groupby(['game_id', 'play_id', 'nfl_id'], as_index=False)
                .tail(1)
                .rename(columns={'frame_id': 'last_frame_id'})
                .reset_index(drop=True)
    )
    
    # Prepare output targets
    output = output_df.copy()
    output = output.rename(columns={'x': 'target_x', 'y': 'target_y'})
    output['id'] = (
        output['game_id'].astype(str) + '_' +
        output['play_id'].astype(str) + '_' +
        output['nfl_id'].astype(str) + '_' +
        output['frame_id'].astype(str)
    )
    
    # Merge observations with targets
    merged = output.merge(
        last_observations, 
        on=['game_id', 'play_id', 'nfl_id'], 
        how='left',
        suffixes=('', '_last')
    )
    
    # Calculate time delta (frames into the future)
    merged['delta_frames'] = (merged['frame_id'] - merged['last_frame_id']).clip(lower=0).astype(float)
    merged['delta_time'] = merged['delta_frames'] / 10.0  # Convert to seconds (10 fps)
    
    return merged

In [15]:
def constant_acceleration_baseline(x_last, y_last, vx_last, vy_last, delta_t, ax_last, ay_last):
    """
    Physics-based baseline: constant acceleration model
    x(t) = x0 + v0*t + 0.5*a*t^2
    """
    pred_x = x_last + vx_last * delta_t + 0.5 * ax_last * (delta_t ** 2)
    pred_y = y_last + vy_last * delta_t + 0.5 * ay_last * (delta_t ** 2)
    
    # Clip to field boundaries
    pred_x = np.clip(pred_x, 0.0, 120.0)
    pred_y = np.clip(pred_y, 0.0, 53.3)
    
    return pred_x, pred_y

In [16]:
def train_ensemble_models(X, y_x, y_y, sample_weights=None, groups=None, 
                         base_x=None, base_y=None, use_lgb=True, use_xgb=True, use_cat=True):
    """
    Train an ensemble of gradient boosting models with cross-validation.
    Uses residual learning: models predict correction to baseline, not absolute position.
    """
    
    models_x_lgb, models_y_lgb = [], []
    models_x_xgb, models_y_xgb = [], []
    models_x_cat, models_y_cat = [], []
    fold_scores = []
    
    # Setup cross-validation
    if USE_GROUP_KFOLD and groups is not None:
        cv = GroupKFold(n_splits=N_FOLDS)
        splits = list(cv.split(X, groups=groups))
    else:
        cv = KFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
        splits = list(cv.split(X))
    
    for fold_idx, (train_idx, val_idx) in enumerate(splits, 1):
        print(f"\n{'='*60}")
        print(f"Fold {fold_idx}/{N_FOLDS} - Train: {len(train_idx):,} | Val: {len(val_idx):,}")
        print(f"{'='*60}")
        
        X_train, X_val = X[train_idx], X[val_idx]
        yx_train, yx_val = y_x[train_idx], y_x[val_idx]
        yy_train, yy_val = y_y[train_idx], y_y[val_idx]
        
        w_train = None if sample_weights is None else sample_weights[train_idx]
        
        # Baseline predictions for this fold
        bx_val = base_x[val_idx] if base_x is not None else np.zeros(len(val_idx))
        by_val = base_y[val_idx] if base_y is not None else np.zeros(len(val_idx))
        
        # --- LightGBM ---
        if use_lgb:
            print("\nTraining LightGBM...")
            train_data_x = lgb.Dataset(X_train, label=yx_train, weight=w_train)
            val_data_x = lgb.Dataset(X_val, label=yx_val, reference=train_data_x)
            
            lgb_x = lgb.train(
                LGBM_PARAMS,
                train_data_x,
                num_boost_round=3000,
                valid_sets=[train_data_x, val_data_x],
                valid_names=['train', 'valid'],
                callbacks=[
                    lgb.early_stopping(stopping_rounds=300, verbose=False),
                    lgb.log_evaluation(period=500)
                ]
            )
            models_x_lgb.append(lgb_x)
            
            train_data_y = lgb.Dataset(X_train, label=yy_train, weight=w_train)
            val_data_y = lgb.Dataset(X_val, label=yy_val, reference=train_data_y)
            
            lgb_y = lgb.train(
                LGBM_PARAMS,
                train_data_y,
                num_boost_round=3000,
                valid_sets=[train_data_y, val_data_y],
                valid_names=['train', 'valid'],
                callbacks=[
                    lgb.early_stopping(stopping_rounds=300, verbose=False),
                    lgb.log_evaluation(period=500)
                ]
            )
            models_y_lgb.append(lgb_y)
            
            pred_x_lgb = lgb_x.predict(X_val)
            pred_y_lgb = lgb_y.predict(X_val)
        
        # --- XGBoost ---
        if use_xgb:
            print("\nTraining XGBoost...")
            dtrain_x = xgb.DMatrix(X_train, label=yx_train, weight=w_train)
            dval_x = xgb.DMatrix(X_val, label=yx_val)
            
            xgb_x = xgb.train(
                XGB_PARAMS,
                dtrain_x,
                num_boost_round=3000,
                evals=[(dtrain_x, 'train'), (dval_x, 'valid')],
                early_stopping_rounds=300,
                verbose_eval=500
            )
            models_x_xgb.append(xgb_x)
            
            dtrain_y = xgb.DMatrix(X_train, label=yy_train, weight=w_train)
            dval_y = xgb.DMatrix(X_val, label=yy_val)
            
            xgb_y = xgb.train(
                XGB_PARAMS,
                dtrain_y,
                num_boost_round=3000,
                evals=[(dtrain_y, 'train'), (dval_y, 'valid')],
                early_stopping_rounds=300,
                verbose_eval=500
            )
            models_y_xgb.append(xgb_y)
            
            pred_x_xgb = xgb_x.predict(dval_x)
            pred_y_xgb = xgb_y.predict(dval_y)
        
        # --- CatBoost ---
        if use_cat:
            print("\nTraining CatBoost...")
            pool_train_x = CatPool(X_train, yx_train, weight=w_train)
            pool_val_x = CatPool(X_val, yx_val)
            
            cat_x = CatBoostRegressor(**CAT_PARAMS)
            cat_x.fit(pool_train_x, eval_set=pool_val_x, verbose=500)
            models_x_cat.append(cat_x)
            
            pool_train_y = CatPool(X_train, yy_train, weight=w_train)
            pool_val_y = CatPool(X_val, yy_val)
            
            cat_y = CatBoostRegressor(**CAT_PARAMS)
            cat_y.fit(pool_train_y, eval_set=pool_val_y, verbose=500)
            models_y_cat.append(cat_y)
            
            pred_x_cat = cat_x.predict(X_val)
            pred_y_cat = cat_y.predict(X_val)
        
        # Ensemble predictions (average)
        pred_x_res = np.zeros(len(val_idx))
        pred_y_res = np.zeros(len(val_idx))
        n_models = 0
        
        if use_lgb:
            pred_x_res += pred_x_lgb
            pred_y_res += pred_y_lgb
            n_models += 1
        if use_xgb:
            pred_x_res += pred_x_xgb
            pred_y_res += pred_y_xgb
            n_models += 1
        if use_cat:
            pred_x_res += pred_x_cat
            pred_y_res += pred_y_cat
            n_models += 1
        
        pred_x_res /= n_models
        pred_y_res /= n_models
        
        # Add back baseline and clip to field
        pred_x_final = np.clip(pred_x_res + bx_val, 0.0, 120.0)
        pred_y_final = np.clip(pred_y_res + by_val, 0.0, 53.3)
        
        # True targets (with baseline added back)
        true_x = yx_val + bx_val
        true_y = yy_val + by_val
        
        # Calculate RMSE
        rmse_x = np.sqrt(mean_squared_error(true_x, pred_x_final))
        rmse_y = np.sqrt(mean_squared_error(true_y, pred_y_final))
        rmse_combined = np.sqrt(0.5 * (rmse_x**2 + rmse_y**2))
        
        print(f"\nFold {fold_idx} Results:")
        print(f"  RMSE X: {rmse_x:.5f}")
        print(f"  RMSE Y: {rmse_y:.5f}")
        print(f"  Combined RMSE: {rmse_combined:.5f}")
        
        fold_scores.append(rmse_combined)
    
    print(f"\n{'='*60}")
    print(f"Cross-Validation Results:")
    print(f"  Per-fold RMSE: {[f'{s:.5f}' for s in fold_scores]}")
    print(f"  Mean RMSE: {np.mean(fold_scores):.5f} ± {np.std(fold_scores):.5f}")
    print(f"{'='*60}")
    
    return {
        'lgb_x': models_x_lgb if use_lgb else None,
        'lgb_y': models_y_lgb if use_lgb else None,
        'xgb_x': models_x_xgb if use_xgb else None,
        'xgb_y': models_y_xgb if use_xgb else None,
        'cat_x': models_x_cat if use_cat else None,
        'cat_y': models_y_cat if use_cat else None,
        'cv_scores': fold_scores
    }

In [ ]:
# Load all training data
train_input, train_output = load_all_train()

Loading training data...


Loading weeks:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
# Apply feature engineering
print("\n" + "="*60)
print("FEATURE ENGINEERING")
print("="*60)

print("\n1. Physics-based features...")
train_input = engineer_physics_features(train_input)

print("2. Formation features...")
train_input = add_formation_features(train_input)

print("3. Temporal features...")
train_input = add_temporal_features(train_input)

print("4. Role features...")
train_input = add_role_features(train_input)

print("5. Neighbor interaction features...")
neighbor_features = compute_neighbor_features(train_input, k_neighbors=K_NEIGHBORS, radius=RADIUS)

print(f"\nFeature engineering complete. Total features: {train_input.shape[1]}")
gc.collect()

In [ ]:
# Create training rows (merge last observations with future targets)
print("\n" + "="*60)
print("CREATING TRAINING DATASET")
print("="*60)

train_df = create_training_rows(train_input, train_output)
print(f"Training rows before merging neighbors: {train_df.shape}")

# Merge neighbor features
train_df = train_df.merge(neighbor_features, on=['game_id', 'play_id', 'nfl_id'], how='left')
print(f"Training rows after merging neighbors: {train_df.shape}")

# Calculate baseline predictions (constant acceleration)
print("\nCalculating baseline (constant acceleration model)...")
baseline_x, baseline_y = constant_acceleration_baseline(
    train_df['x'].values,
    train_df['y'].values,
    train_df['velocity_x'].values,
    train_df['velocity_y'].values,
    train_df['delta_time'].values,
    train_df['acceleration_x'].values,
    train_df['acceleration_y'].values
)

train_df['baseline_x'] = baseline_x
train_df['baseline_y'] = baseline_y

# Calculate residuals (what the model needs to learn)
train_df['residual_x'] = train_df['target_x'] - train_df['baseline_x']
train_df['residual_y'] = train_df['target_y'] - train_df['baseline_y']

# Baseline RMSE
baseline_rmse_x = np.sqrt(mean_squared_error(train_df['target_x'], baseline_x))
baseline_rmse_y = np.sqrt(mean_squared_error(train_df['target_y'], baseline_y))
baseline_rmse = np.sqrt(0.5 * (baseline_rmse_x**2 + baseline_rmse_y**2))

print(f"\nBaseline Performance:")
print(f"  RMSE X: {baseline_rmse_x:.5f}")
print(f"  RMSE Y: {baseline_rmse_y:.5f}")
print(f"  Combined RMSE: {baseline_rmse:.5f}")

gc.collect()

In [ ]:
# Prepare feature matrix
print("\n" + "="*60)
print("PREPARING FEATURE MATRIX")
print("="*60)

# Define feature columns (exclude targets, IDs, etc.)
exclude_cols = ['game_id', 'play_id', 'nfl_id', 'frame_id', 'id', 'last_frame_id',
                'target_x', 'target_y', 'residual_x', 'residual_y',
                'player_name', 'player_birth_date', 'player_position', 
                'player_height', 'player_role', 'player_side', 'play_direction',
                'absolute_yardline_number', 'num_frames_output', 'player_to_predict']

feature_cols = [col for col in train_df.columns if col not in exclude_cols]
print(f"Total features: {len(feature_cols)}")

# Clean data
train_clean = train_df.dropna(subset=['residual_x', 'residual_y'] + feature_cols[:50]).reset_index(drop=True)
print(f"Rows after dropping NaN: {len(train_clean):,}")

# Replace inf with NaN and fill with 0
train_clean[feature_cols] = train_clean[feature_cols].replace([np.inf, -np.inf], np.nan).fillna(0.0)

# Convert to numpy arrays
X = train_clean[feature_cols].astype(np.float32).values
y_x = train_clean['residual_x'].astype(np.float32).values
y_y = train_clean['residual_y'].astype(np.float32).values
baseline_x_arr = train_clean['baseline_x'].astype(np.float32).values
baseline_y_arr = train_clean['baseline_y'].astype(np.float32).values

# Sample weights (prioritize longer horizons)
sample_weights = (1.0 + HORIZON_WEIGHT * train_clean['delta_frames'].clip(1, 5)).astype(np.float32).values

# Groups for GroupKFold (prevent leakage)
groups = (train_clean['game_id'].astype(str) + '_' + 
          train_clean['play_id'].astype(str) + '_' + 
          train_clean['nfl_id'].astype(str)).values

print(f"\nData shapes:")
print(f"  X: {X.shape}")
print(f"  y_x: {y_x.shape}")
print(f"  y_y: {y_y.shape}")
print(f"  Unique groups: {len(np.unique(groups)):,}")

gc.collect()

In [ ]:
# Train ensemble models
print("\n" + "="*60)
print("TRAINING ENSEMBLE MODELS")
print("="*60)

models = train_ensemble_models(
    X, y_x, y_y,
    sample_weights=sample_weights,
    groups=groups,
    base_x=baseline_x_arr,
    base_y=baseline_y_arr,
    use_lgb=True,
    use_xgb=True,
    use_cat=True
)

# Save models
print("\nSaving models...")
with open(SAVE_DIR / 'ensemble_models.pkl', 'wb') as f:
    pickle.dump({
        'models': models,
        'feature_cols': feature_cols,
        'baseline_rmse': baseline_rmse
    }, f)
print(f"Models saved to: {SAVE_DIR / 'ensemble_models.pkl'}")

gc.collect()

In [ ]:
def predict_ensemble(models_dict, X_test, baseline_x, baseline_y):
    """Make predictions using ensemble of models"""
    
    pred_x_res = np.zeros(len(X_test))
    pred_y_res = np.zeros(len(X_test))
    n_models = 0
    
    # LightGBM predictions
    if models_dict['lgb_x'] is not None:
        lgb_pred_x = np.mean([m.predict(X_test) for m in models_dict['lgb_x']], axis=0)
        lgb_pred_y = np.mean([m.predict(X_test) for m in models_dict['lgb_y']], axis=0)
        pred_x_res += lgb_pred_x
        pred_y_res += lgb_pred_y
        n_models += 1
    
    # XGBoost predictions
    if models_dict['xgb_x'] is not None:
        dtest = xgb.DMatrix(X_test)
        xgb_pred_x = np.mean([m.predict(dtest) for m in models_dict['xgb_x']], axis=0)
        xgb_pred_y = np.mean([m.predict(dtest) for m in models_dict['xgb_y']], axis=0)
        pred_x_res += xgb_pred_x
        pred_y_res += xgb_pred_y
        n_models += 1
    
    # CatBoost predictions
    if models_dict['cat_x'] is not None:
        cat_pred_x = np.mean([m.predict(X_test) for m in models_dict['cat_x']], axis=0)
        cat_pred_y = np.mean([m.predict(X_test) for m in models_dict['cat_y']], axis=0)
        pred_x_res += cat_pred_x
        pred_y_res += cat_pred_y
        n_models += 1
    
    # Average ensemble predictions
    pred_x_res /= n_models
    pred_y_res /= n_models
    
    # Add baseline and clip to field boundaries
    pred_x = np.clip(pred_x_res + baseline_x, 0.0, 120.0)
    pred_y = np.clip(pred_y_res + baseline_y, 0.0, 53.3)
    
    return pred_x, pred_y

In [ ]:
# Load test data
print("\n" + "="*60)
print("PREPARING TEST DATA")
print("="*60)

test_input = pd.read_csv(BASEDIR / 'test_input.csv')
test_template = pd.read_csv(BASEDIR / 'test.csv')

print(f"Test input shape: {test_input.shape}")
print(f"Test template shape: {test_template.shape}")

# Apply same feature engineering to test data
print("\nApplying feature engineering to test data...")
test_input = engineer_physics_features(test_input)
test_input = add_formation_features(test_input)
test_input = add_temporal_features(test_input)
test_input = add_role_features(test_input)

print("Computing neighbor features for test data...")
neighbor_features_test = compute_neighbor_features(test_input, k_neighbors=K_NEIGHBORS, radius=RADIUS)

gc.collect()

In [ ]:
# Prepare test features
print("\nPreparing test features...")

# Get last frame observations
test_last = (test_input.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
                       .groupby(['game_id', 'play_id', 'nfl_id'], as_index=False)
                       .tail(1)
                       .rename(columns={'frame_id': 'last_frame_id'}))

# Merge with test template
test_df = test_template.merge(test_last, on=['game_id', 'play_id', 'nfl_id'], how='left')
test_df = test_df.merge(neighbor_features_test, on=['game_id', 'play_id', 'nfl_id'], how='left')

# Calculate time deltas
test_df['delta_frames'] = (test_df['frame_id'] - test_df['last_frame_id']).clip(lower=0).astype(float)
test_df['delta_time'] = test_df['delta_frames'] / 10.0

# Calculate baseline predictions for test
baseline_x_test, baseline_y_test = constant_acceleration_baseline(
    test_df['x'].values,
    test_df['y'].values,
    test_df['velocity_x'].values,
    test_df['velocity_y'].values,
    test_df['delta_time'].values,
    test_df['acceleration_x'].values,
    test_df['acceleration_y'].values
)

# Ensure all feature columns exist
for col in feature_cols:
    if col not in test_df.columns:
        test_df[col] = 0.0

# Clean test features
test_df[feature_cols] = test_df[feature_cols].replace([np.inf, -np.inf], np.nan).fillna(0.0)

X_test = test_df[feature_cols].astype(np.float32).values

print(f"Test feature matrix shape: {X_test.shape}")

gc.collect()

In [ ]:
# Make predictions
print("\n" + "="*60)
print("GENERATING PREDICTIONS")
print("="*60)

pred_x, pred_y = predict_ensemble(models, X_test, baseline_x_test, baseline_y_test)

# Create submission
submission = pd.DataFrame({
    'id': (test_df['game_id'].astype(str) + '_' +
           test_df['play_id'].astype(str) + '_' +
           test_df['nfl_id'].astype(str) + '_' +
           test_df['frame_id'].astype(str)),
    'x': pred_x,
    'y': pred_y
})

# Save submission
submission_path = SAVE_DIR / 'submission.csv'
submission.to_csv(submission_path, index=False)

print(f"\nSubmission saved to: {submission_path}")
print(f"Submission shape: {submission.shape}")
print(f"\nFirst few predictions:")
print(submission.head(10))

print("\n" + "="*60)
print("COMPLETE!")
print("="*60)

## Summary and Next Steps

### Key Improvements Implemented:

1. **Enhanced Feature Engineering**:
   - Physics-based features (momentum, kinetic energy, force projections)
   - Ball-relative coordinate system (parallel/perpendicular velocities)
   - Temporal patterns (lag features, rolling statistics, trajectory curvature)
   - Neighbor interactions (weighted aggregation of nearby players)
   - Formation features (team centroids, spatial relationships)

2. **Multi-Model Ensemble**:
   - LightGBM (fast, efficient gradient boosting)
   - XGBoost (robust, handles complex patterns)
   - CatBoost (strong baseline performance)
   - Average ensemble for robustness

3. **Residual Learning**:
   - Baseline: constant acceleration physics model
   - Models learn to correct baseline errors
   - More stable and interpretable than direct position prediction

4. **Advanced Validation**:
   - GroupKFold to prevent data leakage
   - Horizon-weighted loss (prioritize longer predictions)
   - Time-aware splitting

### Expected Performance:
- The baseline constant acceleration model achieves ~1.7-2.0 RMSE
- Our ensemble approach should improve this to **~1.3-1.5 RMSE** or better
- The multi-model ensemble provides robustness against overfitting

### Potential Further Improvements:
1. **Neural Network Component**: Add LSTM/Transformer for sequence modeling
2. **Role-Specific Models**: Train separate models for QB, WR, DB, etc.
3. **Play Context**: Incorporate down, distance, score differential
4. **Advanced Physics**: Model player deceleration, reaction times
5. **Hyperparameter Tuning**: Bayesian optimization for each model
6. **Stacking**: Use meta-learner to combine model predictions